In [3]:
import numpy as np
import pandas as pd

import re
import html
import requests

from datetime import datetime, date
import plotly.express as px
import plotly.graph_objs as go



def Q1(row):
    if (row['Q1'] == 1.0) or (row['Q1_Pred'] == 1.0):
        return 1.0
    else:
        return 0.0
    
def Q2(row):
    if (row['Q2'] == 1.0) or (row['Q2_Pred'] == 1.0):
        return 1.0
    else:
        return 0.0

def Q3(row):
    if (row['Q3'] == 1.0) or (row['Q3_Pred'] == 1.0):
        return 1.0
    else:
        return 0.0
    
def Q4(row):
    if (row['Q4'] == 1.0) or (row['Q4_Pred'] == 1.0):
        return 1.0
    else:
        return 0.0




def q(row):
    if row['Q1_fin'] == 1.0:
        return 'Q1'
    if row['Q2_fin'] == 1.0:
        return 'Q2'
    if row['Q3_fin'] == 1.0:
        return 'Q3'
    if row['Q4_fin'] == 1.0:
        return 'Q4'
    else:
        return 'unk'

def p(row):
    if row['Q1_fin'] == 1.0:
        if row['Q1_Prob'] > 0.0:
            return row['Q1_Prob']
        else:
            return 'Given'
    if row['Q2_fin'] == 1.0:
        if row['Q2_Prob'] > 0.0:
            return row['Q2_Prob']
        else:
            return 'Given'
    if row['Q3_fin'] == 1.0:
        if row['Q3_Prob'] > 0.0:
            return row['Q3_Prob']
        else:
            return 'Given'
    if row['Q4_fin'] == 1.0:
        if row['Q4_Prob'] > 0.0:
            return row['Q4_Prob']
        else:
            return 'Given'
    else:
        return 'unk'

def unk(row):
    if row['Q'] == 'unk':
        return 1.0
    else:
        return 0.0


def buildout():
    """[Provides a final set of features showing which salary range applies to each data.]

    Returns:
        [csv]: [a table containing our original original, predicted, and bracket columns]
    """
    ### changed from app/
    df = pd.read_csv('../app/data/ml.csv')
    df['DatePosted'] = df['DatePosted'].astype('datetime64')
    df = df.sort_values(by="DatePosted")


    df.replace(np.nan, 0, inplace = True)
    df['Q1_fin'] = df.apply( lambda row : Q1(row), axis = 1)
    df['Q2_fin'] = df.apply( lambda row : Q2(row), axis = 1)
    df['Q3_fin'] = df.apply( lambda row : Q3(row), axis = 1)
    df['Q4_fin'] = df.apply( lambda row : Q4(row), axis = 1)
    df['Q'] = df.apply( lambda row : q(row), axis = 1)
    df['P'] = df.apply( lambda row : p(row), axis = 1)
    df['unk'] = df.apply( lambda row : unk(row), axis = 1)
    
    df.to_csv(f'../app/data/graph_ready.csv', index=False)
    return df

In [28]:
df = buildout()

In [29]:
len(df)

6732

In [17]:
df = df[['JobTitle','Company','State','State','Summary','Salary','Q1_Pred',
       'Q1_Prob', 'Q2_Pred', 'Q2_Prob', 'Q3_Pred', 'Q3_Prob', 'Q4_Pred',
       'Q4_Prob', 'Q1_fin', 'Q2_fin', 'Q3_fin', 'Q4_fin', 'Q', 'P', 'unk']]

In [27]:
df[6380:]

,JobTitle,Company,State,State,Summary,Salary,Q1_Pred,Q1_Prob,Q2_Pred,Q2_Prob,...,Q3_Prob,Q4_Pred,Q4_Prob,Q1_fin,Q2_fin,Q3_fin,Q4_fin,Q,P,unk
6137,data analysts (r/rstudio),"asap solutions group, llc",Remote,Remote,master’s degree in public health epidemiology...,93857.142857,0.0,0.00,0.0,0.00,...,0.00,0.0,0.00,0.0,1.0,0.0,0.0,Q2,Given,0.0
6179,machine learning engineer,clara analytics,Remote,Remote,5 years experience at a software company as a...,150000.000000,0.0,0.00,0.0,0.00,...,0.00,0.0,0.00,0.0,0.0,0.0,1.0,Q4,Given,0.0
6142,"part-time, content writer",kaizen analytix,Remote,Remote,4 years of experience in professional creative...,62571.428571,0.0,0.00,0.0,0.00,...,0.00,0.0,0.00,1.0,0.0,0.0,0.0,Q1,Given,0.0
6144,azure data engineer,bicp,Remote,Remote,designs and develops data warehouse data anal...,166857.142857,0.0,0.00,0.0,0.00,...,0.00,0.0,0.00,0.0,0.0,0.0,1.0,Q4,Given,0.0
6174,principal data scientist,ghk,Remote,Remote,leads ai best practice sharing opportunities a...,120000.000000,0.0,0.00,0.0,0.00,...,0.00,0.0,0.00,0.0,0.0,1.0,0.0,Q3,Given,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,scientist (pcr assay validation),"clear gene, inc.",CA,CA,we combine advances in data science with rigor...,0.000000,0.0,1.00,1.0,0.63,...,1.00,0.0,0.88,0.0,1.0,0.0,0.0,Q2,0.63,0.0
350,"senior associate, machine learning engineer",capital one - us,MA,MA,master's or doctoral degree in computer scienc...,0.000000,0.0,1.00,0.0,0.77,...,0.96,0.0,0.78,0.0,0.0,0.0,0.0,unk,unk,1.0
363,sales operations analyst,omniscient neurotechnology,GA,GA,experience with hubspot or another crm is a pl...,0.000000,0.0,0.88,0.0,0.77,...,1.00,0.0,0.96,0.0,0.0,0.0,0.0,unk,unk,1.0
352,consumer & community banking - card risk machi...,"jpmorgan chase bank, n.a.",DE,DE,assess the effectiveness and accuracy of new d...,0.000000,0.0,1.00,0.0,0.82,...,0.89,0.0,0.76,0.0,0.0,1.0,0.0,Q3,0.89,0.0
